In [1]:
import os
import re
import pickle
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
from urllib.request import urlopen
from io import BytesIO
from PIL import Image
import colour
import numpy as np

In [3]:
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [4]:
cookieFile = "cookie.pkl"
cookies = None

if os.path.exists(cookieFile):
    with open(cookieFile, "rb") as inFile: cookies = pickle.load(inFile)

In [5]:
driver = webdriver.Firefox()
if cookies is not None:
    driver.get("https://weibo.com")
    for co in cookies: driver.add_cookie(co)
driver.get("https://weibo.com")
# _ = input("press ENTER after login to weibo")

In [6]:
# 跳转到搜索“腾讯”官方号
driver.get(r"https://s.weibo.com/user?q=%E8%85%BE%E8%AE%AF&auth=org_vip&page=1")

In [16]:
# 跳转到搜索“电竞”官方号
#driver.get(r"https://s.weibo.com/user?q=%E7%94%B5%E7%AB%9E&auth=org_vip&Refer=SUer_box")

In [ ]:
# 跳转到搜索“视频”官方号
#driver.get(r"https://s.weibo.com/user?q=%E8%85%BE%E8%AE%AF&auth=org_vip&page=1")

In [17]:
def wait_for_element(driver, className, timeout=30):
    try:
        el = EC.presence_of_element_located((By.CLASS_NAME, className))
        WebDriverWait(driver, timeout).until(el)
    except TimeoutException:
        print("Timeout")
        return False
    finally:
        return True

In [18]:
def collect_color_temperature(driver):
    url = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "body.S_page"))).value_of_css_property("background-image")
    if "none" in url: return None
    imgurl = re.split('[""]', url)[1]
    image = np.array(Image.open(BytesIO(urlopen(imgurl).read())).convert("RGB"))
    imageXYZ = np.mean(np.mean((image / 255.0), axis=0), axis=0)
    imageXY = colour.XYZ_to_xy(imageXYZ)
    CCT = colour.temperature.xy_to_CCT_McCamy1992(imageXY)
    return CCT

In [19]:
def collect_search(driver, url, num_pages=5):
    driver.get(url)
    result = []
    for i in range(num_pages):
        page = []
        if not wait_for_element(driver, "name"): break
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        for name in soup.select("div.info a.name"):
            page.append((name.get_text(), name["href"]))
        result.append(page)
        try:
            nextPageButton = driver.find_element_by_xpath('//a[text()="下一页"]')
            nextPageButton.click()
        except Exception as e:
            print(e)
            break
    return result

In [20]:
def collect_save(driver,searchData):
    result = []
    for page in searchData:
        for element in page:
            driver.get("https:"+element[1])
            if not wait_for_element(driver, "PCD_counter"): continue
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            eleThree = soup.select("div#plc_main div.PCD_counter td.S_line1")
            if len(eleThree) != 3: continue #确认搜到的是3个element
            row = [
                element[0], # 名字
                "https:"+element[1], # url
                int(eleThree[0].strong.string), # 关注数
                int(eleThree[1].strong.string), # 粉丝数
                int(eleThree[2].strong.string), # 微博数
                collect_color_temperature(driver) # 色温
            ]
            result.append(row)
    result = pd.DataFrame(result,columns = ["名字","url","关注数","粉丝数","微博数","色温"])
    return result

In [21]:
tmp = collect_search(driver, r"https://s.weibo.com/user?q=%E7%94%B5%E7%AB%9E&auth=org_vip&Refer=SUer_box", num_pages=3)

In [22]:
data = collect_save(driver, tmp)

In [23]:
data.to_csv("Esport.csv", index=False, encoding='utf-8-sig')

In [24]:
data

,名字,url,关注数,粉丝数,微博数,色温
0,新浪体育电竞,https://weibo.com/u/1665680640,423,1308565,18783,492904.465730
1,微博电竞,https://weibo.com/u/6249241592,839,2632788,6240,NaN
2,新浪电竞,https://weibo.com/haiqing1988,62,723492,1804,9591.902915
3,PentaQ刺猬电竞社,https://weibo.com/LOLPentaQ,1022,393695,48104,3202.147801
4,腾讯电竞,https://weibo.com/u/6062063357,1259,1437393,10619,3932.712282
5,人民电竞,https://weibo.com/renmindianjing,165,390174,2644,5458.951966
6,微博电竞全明星战队,https://weibo.com/u/7504362536,55,338090,125,4808.133961
7,RW电竞俱乐部,https://weibo.com/RogueWarriors,201,921279,1797,3558.963211
8,AutoFull傲风电竞椅,https://weibo.com/AutoFull,1300,365641,9646,4940.698144
9,AORUS技嘉电竞,https://weibo.com/gigabytevga,282,1002148,5503,8425.019727


In [25]:
driver.get("https://weibo.com")
with open(cookieFile, "wb") as outFile:
    pickle.dump(driver.get_cookies(), outFile)

driver.close()

WebDriverException: Message: Failed to decode response from marionette
